In [ ]:
! pip install transformers


ERROR: Could not find a version that satisfies the requirement layoutlm (from versions: none)
ERROR: No matching distribution found for layoutlm


In [ ]:
from PIL import Image, ImageDraw, ImageFont

labels = ["Total", "Other"]
label_map = {i: label for i, label in enumerate(labels)}
type(label_map)


dict

In [ ]:
from transformers import LayoutLMModel, LayoutLMConfig, LayoutLMTokenizer

# Define Tokenizer
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")


In [ ]:
from torch.utils.data import Dataset, DataLoader

# Define pytorch Dataset

class InvoiceDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir # Directory with all the images
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.toList()
        
def normalize_bbox(bbox, width, height):
     # Width and height correspond to those of the original image
     return [
         int(1000 * (bbox[0] / width)),
         int(1000 * (bbox[1] / height)),
         int(1000 * (bbox[2] / width)),
         int(1000 * (bbox[3] / height)),
     ]



In [ ]:
from transformers import LayoutLMForTokenClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(labels))
model.to(device)

In [ ]:
# Try forward pass with a single example
sequence_length = 500
bbox = [160, 301, 58, 113]
content = "TXU"
label = "Other"

# bbox_pad = torch.zeros(4)
bbox = normalize_bbox([160,301,58,13], 2000, 2600)
bbox=[[[1,2,3,5], [1,2,45,2], [6,5,42,1], [7,52,12,9]]]

bbox = torch.tensor(bbox, dtype=torch.long)
# bbox_pad[:4] = bbox
print(bbox)
temp = tokenizer(content)
print(tokenizer.decode(temp['input_ids']))
input_ids = torch.tensor([temp['input_ids']], dtype=torch.long)
token_type_ids = torch.tensor([temp['token_type_ids']], dtype=torch.long)
labels = torch.tensor([[1, 0, 0, 0]])
print(labels.shape)
attention_mask = torch.tensor([temp['attention_mask']], dtype=torch.long)
print(input_ids, token_type_ids, labels, attention_mask)

print(bbox.shape, input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
# out = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
# out = model(input_ids = input_ids, labels = labels)
print(out)

tensor([[[ 1,  2,  3,  5],
         [ 1,  2, 45,  2],
         [ 6,  5, 42,  1],
         [ 7, 52, 12,  9]]])
[CLS] txu [SEP]
torch.Size([1, 4])
tensor([[  101, 19067,  2226,   102]]) tensor([[0, 0, 0, 0]]) tensor([[1, 0, 0, 0]]) tensor([[1, 1, 1, 1]])
torch.Size([1, 4, 4]) torch.Size([1, 4]) torch.Size([1, 4]) torch.Size([1, 4]) torch.Size([1, 4])
TokenClassifierOutput(loss=tensor(0.6273, grad_fn=<NllLossBackward>), logits=tensor([[[ 0.0989,  0.3598],
         [ 0.5079, -0.0168],
         [ 0.2693,  0.1612],
         [ 0.0984,  0.3603]]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)


In [ ]:
from transformers import LayoutLMTokenizer, LayoutLMModel
import torch
tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMModel.from_pretrained('microsoft/layoutlm-base-uncased')
words = ["worthless", "world"]
normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
token_boxes = []
for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
# add bounding boxes of cls + sep tokens
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]
encoding = tokenizer(' '.join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids)
last_hidden_states = outputs.last_hidden_state
print(bbox, input_ids, attention_mask)

tensor([[[   0,    0,    0,    0],
         [ 637,  773,  693,  782],
         [ 698,  773,  733,  782],
         [1000, 1000, 1000, 1000]]]) tensor([[  101, 22692,  2088,   102]]) tensor([[1, 1, 1, 1]])


In [ ]:
# Training

from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 5
t_total = len(train_dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
  for batch in tqdm(train_dataloader, desc="Training"):
      input_ids = batch[0].to(device)
      bbox = batch[4].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)
      labels = batch[3].to(device)

      # forward pass
      outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                      labels=labels)
      loss = outputs.loss

      # print loss every 100 steps
      if global_step % 100 == 0:
        print(f"Loss after {global_step} steps: {loss.item()}")

      # backward pass to get the gradients 
      loss.backward()

      #print("Gradients on classification head:")
      #print(model.classifier.weight.grad[6,:].sum())

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

# Legacy Code

In [ ]:
coordinates = [50, 80, 70, 100]
sequence = "total"
